In [1]:
import numpy as np
import spacy
import tqdm
from torch.utils.data import Dataset
import pandas
import torch
import sklearn.metrics
import torch.utils.data

c:\Users\dollj\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


 That one hot + neural network model did not work very well!
 So now for a different
 technique that treats text as a sequence,
 this will involve recurrent
 networks, using a particular kind called an LSTM.

In [2]:
nlp = spacy.load('en_core_web_lg')

 Using the same dataset of sentiments on movie reviews, we
 will use a pre-trained language model from spacy.
 Using wikipedia, spacy comes pretrained with word vectors,
 which are dense encodings, so instead of one hot encoding,
 we use the word vector.
 The nice thing about this is we actually do less
 work to set up our data AND our model starts
  with knowledge from the language
 model built over wikipedia.
 Here is an example word vector:

In [3]:
for token in nlp('hello'):
    print(token)
    print(token.vector)
    print(token.vector.shape)

hello
[ 2.2407e+00  1.0389e+00  1.3092e+00 -1.7335e+00 -7.8466e-01 -2.9269e-01
 -1.8059e+00 -2.5223e+00  7.8025e-01  2.4899e+00 -9.1849e-02  2.8755e-01
 -1.5057e+00  2.6337e+00  2.5252e+00 -2.2432e-01 -2.2068e+00 -5.7895e-01
 -5.6551e-01 -1.9338e+00  1.4973e+00  8.5889e-01  3.3559e+00 -3.7527e+00
  2.2585e-01 -1.6969e-01  5.1389e-01  4.6073e-01 -2.8248e-01 -2.6048e+00
 -3.5896e+00 -1.0971e+00 -1.5517e+00 -1.2185e-01  2.8633e+00 -1.2525e+00
 -1.6924e+00 -2.2917e+00  9.7793e-01  4.6954e-01 -3.5950e+00 -1.7357e-01
  9.8050e-01 -1.8044e+00 -7.2183e-01 -4.0709e-01 -3.0943e+00  1.3095e-01
 -2.9015e+00  1.4768e+00 -1.0588e+00 -2.8123e+00  1.2936e+00 -7.5977e-03
  2.9975e+00 -2.4438e+00  1.2348e-01  1.8322e+00  3.5869e-01 -1.8335e-02
  1.9534e+00  1.4417e+00  9.9895e-01 -2.8209e+00 -7.5846e-01 -1.8438e+00
 -3.2658e+00 -4.6574e-01  9.0322e-01  7.9868e-01 -1.6134e+00 -3.3082e-01
  1.1541e+00 -4.7334e+00  1.4964e+00 -2.4014e+00 -1.3461e+00 -9.5551e-01
  2.9671e-01 -1.4506e+00 -8.7128e-01 -3.0714e

 And for the dataset, we just extract the vectors as tensors
 and return the length of each string in tokens.
 This is important for working with pytorch recurrent networks.

In [4]:
class SentimentDataset(Dataset):
    def __init__(self):
        self.data = pandas \
            .read_csv('C:\github\pytorch-dl7\sentiment.tsv', sep='\t', header=0) \
            .groupby('SentenceId') \
            .first()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if type(idx) is torch.Tensor:
            idx = idx.item()
        sample = self.data.iloc[idx]
        token_vectors = []
        # switching off NER for a tiny speed boost
        for token in nlp(sample.Phrase.lower(), disable=['ner']):
            token_vectors.append(token.vector)

        # tokens and length as inputs -- the length
        # is needed to 'pack' variable length sequences
        # output is the sentiment score 
        return (torch.tensor(token_vectors),
                torch.tensor(len(token_vectors)),
                torch.tensor(sample.Sentiment))


sentiment = SentimentDataset()
sentiment[0]

<ipython-input-4-e8c5ee0f4013>:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:210.)
  return (torch.tensor(token_vectors),


(tensor([[ -9.3629,   9.2761,  -7.2708,  ...,   2.6801,  -6.8160,   3.5737],
         [ -2.9056,  -2.5910,   3.3317,  ...,  -7.2864,  -3.5043,  -2.2919],
         [-12.6670,  -6.5680,  -0.6154,  ...,  -8.0021,  -0.3171,  -7.7062],
         ...,
         [ -9.3629,   9.2761,  -7.2708,  ...,   2.6801,  -6.8160,   3.5737],
         [  2.7879,  -2.6493,  -0.4601,  ...,  -0.6575,  -5.5132,   0.4322],
         [ -0.0765,  -4.6896,  -4.0431,  ...,   1.3040,  -0.5270,  -1.3622]]),
 tensor(37),
 tensor(1))

In [5]:
# break this into a training and testing dataset, and need
# to collate into fixed width as these will be
# variable batches
def collate(batch):
    # sort indescending length order -- this is needed for
    # padding seqeunces in pytorch
    batch.sort(key=lambda x: x[1], reverse=True)
    sequences, lengths, sentiments = zip(*batch)
    sequences = torch.nn.utils.rnn.pad_sequence(
        sequences, batch_first=True
    )
    sentiments = torch.stack(sentiments)
    lengths = torch.stack(lengths)
    return sequences, lengths, sentiments


number_for_testing = int(len(sentiment) * 0.05)
number_for_training = len(sentiment) - number_for_testing
train, test = torch.utils.data.random_split(sentiment,
                                            [number_for_training, number_for_testing])
trainloader = torch.utils.data.DataLoader(
    train, batch_size=32, shuffle=True,
    collate_fn=collate)
testloader = torch.utils.data.DataLoader(
    test, batch_size=32, shuffle=True,
    collate_fn=collate)

# take a peek and see what we are collating
for batch in trainloader:
    print(batch[0].shape, batch[1].shape, batch[2].shape)
    # what is the max length?
    print(batch[1][0])
    break

torch.Size([32, 38, 300]) torch.Size([32]) torch.Size([32])
tensor(38)


 Now, this is still a regression problem, but instead of
 one hot encoded words and a plain nerual network,
 we will have sequences
 of word vectors, from the learned wikipedia model.
 These sequences in turn will be * packed * which
 is because they all have different lengths,
 run through the recurrent network
 which loops word vector by word vector to compute a final
 numerical representation of the whole sequence - -just like
 reading - -word for word in order.
 This is why we need the sequence lengths, you need to know
 the boundaries on which to pack.

In [6]:
class Model(torch.nn.Module):

    def __init__(self, input_dimensions, size=128, layers=1):
        super().__init__()
        self.seq = torch.nn.LSTM(input_dimensions, size, layers)
        self.layer_one = torch.nn.Linear(size * layers, size)
        self.activation_one = torch.nn.ReLU()
        self.layer_two = torch.nn.Linear(size, size)
        self.activation_two = torch.nn.ReLU()
        self.shape_outputs = torch.nn.Linear(size, 5)

    def forward(self, inputs, lengths):
        # need to sort the sequences for pytorch -- which we
        # did in our collation above
        number_of_batches = lengths.shape[0]
        packed_inputs = torch.nn.utils.rnn.pack_padded_sequence(
            inputs,
            lengths,
            batch_first=True)
        buffer, (hidden, cell) = self.seq(packed_inputs)
        # batch first...
        buffer = hidden.permute(1, 0, 2)
        # flatten out the last hidden state -- this will
        # be the tensor representing each batch
        buffer = buffer.contiguous().view(number_of_batches, -1)
        # and feed along to a simple output network with
        # a single output cell for regression
        buffer = self.layer_one(buffer)
        buffer = self.activation_one(buffer)
        buffer = self.layer_two(buffer)
        buffer = self.activation_two(buffer)
        buffer = self.shape_outputs(buffer)
        return buffer


# get the input dimensions from the first sample
# encodings are word, vectors - so index 1 at the end
model = Model(sentiment[0][0].shape[1])

In [7]:
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()
model.train()
for epoch in range(64):
    losses = []
    for sequences, lengths, sentiments in tqdm.tqdm(trainloader):
        optimizer.zero_grad()
        results = model(sequences, lengths)
        loss = loss_function(results, sentiments)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("Loss: {0}".format(torch.tensor(losses).mean()))

100%|██████████| 254/254 [00:46<00:00,  5.51it/s]


Loss: 1.4734059572219849


100%|██████████| 254/254 [00:45<00:00,  5.60it/s]


Loss: 1.3330905437469482


100%|██████████| 254/254 [00:46<00:00,  5.49it/s]


...

Loss: 0.05004001408815384


100%|██████████| 254/254 [00:47<00:00,  5.31it/s]


Loss: 0.06291337311267853


100%|██████████| 254/254 [00:47<00:00,  5.31it/s]

Loss: 0.11179642379283905


In [8]:
results_buffer=[]
actual_buffer=[]
with torch.no_grad():
    model.eval()
    for test_seq, test_len, test_sentiment in testloader:
        results=model(test_seq, test_len).argmax(dim=1).numpy()
        results_buffer.append(results)
        actual_buffer.append(test_sentiment)

print(sklearn.metrics.classification_report(
    np.concatenate(actual_buffer),
    np.concatenate(results_buffer)))

              precision    recall  f1-score   support

           0       0.35      0.33      0.34        48
           1       0.44      0.43      0.43       112
           2       0.32      0.31      0.31        91
           3       0.42      0.48      0.45       112
           4       0.51      0.43      0.47        63

    accuracy                           0.41       426
   macro avg       0.41      0.40      0.40       426
weighted avg       0.41      0.41      0.41       426

